<a href="https://colab.research.google.com/github/BrunoBmassa/Python-Actitivies/blob/main/agregacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [71]:
import pyspark
import gdown
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, var_pop, format_number, min, max, first, last, count, count_distinct, asc, sum, col

In [65]:
spark= SparkSession.builder.getOrCreate()

In [ ]:
#Leia o arquivo 'videos-preparados.snappy.parquet' no dataframe 'df_video'

df_video_url='https://drive.google.com/file/d/1A1iKcyCfaa0ObhTVN-3EfhiALnB0aCN0/view?usp=drive_link'
# Extração do arquivo pela URL
file_id = df_video_url.split('/')[-2]
# Download do arquivo
output_file = 'videos-preparados.snappy.parquet'
gdown.download(id=file_id, output=output_file, quiet=False)
#Leitura do arquivo inferindo o esquema
df_video = spark.read.parquet(output_file)


In [ ]:
#Calcule a quantidade de registros para cada valor único da coluna "Keyword"

df_video.groupBy("Keyword").count().show()

In [ ]:
#Calcule a média da coluna "Interaction" para cada valor único da coluna 'Keyword'

df_video.groupBy("Keyword").avg("Interaction").show()


In [ ]:
#Calcule o valor máximo da coluna "Interaction" para cada valor único da coluna 'Keyword' e nomeie de 'Rank Interactions', em seguida ordene pela nova coluna em ordem decrescente

df_video.groupBy("Keyword").max("Interaction").withColumnRenamed("max(Interaction)", "Rank Interactions").orderBy("Rank Interactions", ascending=False).show()


In [ ]:
#Calcule a média e a variância da coluna 'Views' para cada valor único da coluna 'Keyword'

df_video.groupBy("Keyword").agg(avg("Views"), var_pop("Views")).show()


In [ ]:
#Calcule a média, o valor mínimo e o valor máximo de 'Views' para cada valor único da coluna 'Keyword', sem casas decimais

df_video.groupBy("Keyword").agg(
    format_number(avg("Views"), 2).alias("avg_views"),
    format_number(min("Views"), 2).alias("min_views"),
    format_number(max("Views"), 2).alias("max_views")
).show()


In [ ]:
#Mostre o primeiro e o último 'Published At' para cada valor único da coluna 'Keyword'

df_video.groupBy("Keyword").agg(first("Published At"), last("Published At")).show()

In [ ]:
#Conte todos os 'title' de forma normal e todos os únicos e verifique se há diferença

df_video.groupBy("title").agg(
      count("title").alias("Contagem normal"),
      count_distinct("title").alias("Contagem unicos")
      ).show()


In [ ]:
#Mostre a quantidade de registros ordenados por ano em ordem ascendente

df_video.groupBy(df_video["Published At"].substr(1, 4).alias("Ano")).count().orderBy("Ano").show()


In [ ]:
#Mostre a quantidade de registros ordenados por ano e mês em ordem ascendente

df_video.groupBy(df_video["Published At"].substr(1, 4).alias("Ano"),
                 df_video["Published At"].substr(6, 2).alias("Mes")).count().orderBy("Ano", "Mes").show()

In [ ]:
#Calcule a média acumulativa de ‘Likes’ para cada ‘Keyword’ ao longo dos anos

df_video = df_video.withColumn("Ano", df_video["Published At"].substr(1, 4))
window_spec = Window.partitionBy("Keyword").orderBy("Ano").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_video = df_video.withColumn("Media Acumulada Likes", format_number(avg("Likes").over(window_spec), 0))
df_video.show()
